In [1]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
if "SUPABASE_URL" not in os.environ:
    os.environ["SUPABASE_URL"] = getpass.getpass("Supabase URL:")
if "SUPABASE_SERVICE_KEY" not in os.environ:
    os.environ["SUPABASE_SERVICE_KEY"] = getpass.getpass("Supabase Service Key:")

In [2]:
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
import json

with open("../prereqMap.json", "r") as f:
    prereq_map = json.load(f)

# --- Fetch all course prerequisite texts from Supabase ---
def fetch_prerequisite_texts():
    response = supabase.table("courses_new").select("course_name", "prerequisites").execute()
    rows = response.data
    return {row["course_name"]: row.get("prerequisites", "") for row in rows}

text_prereqs = fetch_prerequisite_texts()

In [ ]:
# import openai
# import time
# def needs_grad_req_gpt(prereq_text):
#     prompt = f"""
#         The following course has this prerequisite text:

#         "{prereq_text}"

#         Based on this, can an undergraduate take the course (OR do they strictly need to be a graduate/professional-level student?)
#         Examples:
#         "Graduate/professional standing or declared in Statistics VISP" : NO
#         "Graduate/professional standing or declared in Mathematics VISP": NO
#         "Graduate/professional standing andF&WECTAT 572orAN SCI 865": NO
#         "Graduate/professional standing or (PHYSICS 202or208andZOOLOGY/​BIOLOGY/​BOTANY  152or153)": YES
#         "Graduate/professional standing or Declared in Analytics for Decision Making capstone certificate": YES
#         "JOURN 335,345, or graduate/professional standing": YES

#         Return your answer as either YES or NO. YES means an undergrad CAN take the course, and NO means an undergrad cannot take the course.
#         """
#     res = openai.chat.completions.create(
#         model="gpt-4.1-nano",
#         messages=[{"role": "user", "content": prompt}],
#         temperature=0
#     )
#     time.sleep(0.5)
#     return "no" in res.choices[0].message.content.lower()


In [126]:
def is_grad_only_text(prereq_text: str) -> bool:
    prereq_text = prereq_text.lower().strip()

    if "graduate/professional standing" in prereq_text and "or" not in prereq_text:
        return True
    for phrase in [
        "graduate standing and",
        "must be a graduate student",
        "open only to graduate",
        "graduate or dissertator",
        "professional standing and",
        "visp (graduate",
        "graduate-level visp"
    ]:
        if phrase in prereq_text:
            return True
    return False

In [ ]:
# violations: COMARTS 613
#PhD


In [127]:
grad_requirement_cache = {}

def requires_grad_standing(course, prereq_map, text_prereqs, visited=None):
    if visited is None:
        visited = set()
    if course in grad_requirement_cache:
        if course == "COMARTS 613":
            print("1", grad_requirement_cache[course])
        return grad_requirement_cache[course]
    if course in visited:
        if course == "COMARTS 613":
            print("2")
        return False
    visited.add(course)

    prereq_text = text_prereqs.get(course, "").lower().strip()
    if prereq_text == "none":
        if course == "COMARTS 613":
            print("3")
        grad_requirement_cache[course] = False
        return False

    if is_grad_only_text(prereq_text):
        if course == "COMARTS 613":
            print("4")
        grad_requirement_cache[course] = True
        return True

    if prereq_map.get(course, []) == []:
        return False
    
    for child in prereq_map.get(course, []):
        if not requires_grad_standing(child, prereq_map, text_prereqs, visited):
            if course == "COMARTS 613":
                print("5")
            grad_requirement_cache[course] = False
            return False
    if course == "COMARTS 613":
            print("6")

    grad_requirement_cache[course] = True
    return True



In [128]:
updates = []
for course in prereq_map:
    grad_only = requires_grad_standing(course, prereq_map, text_prereqs)
    updates.append({"course_name": course, "needs_grad_standing": grad_only})

c = requires_grad_standing("COMARTS 613", prereq_map, text_prereqs)
print(c)
# Upsert into Supabase


5
1 False
False


In [ ]:
# BATCH_SIZE = 100  # Start with 100 and increase if stable

# already_inserted = True
# if not already_inserted:
# for i in range(0, len(updates), BATCH_SIZE):
#     batch = updates[i:i + BATCH_SIZE]
#     supabase.table("courses_new") \
#         .upsert(batch, on_conflict=["course_name"]) \
#         .execute()
#     print(f"Completed batch: {i}")

Completed batch: 0
Completed batch: 100
Completed batch: 200
Completed batch: 300
Completed batch: 400
Completed batch: 500
Completed batch: 600
Completed batch: 700
Completed batch: 800
Completed batch: 900
Completed batch: 1000
Completed batch: 1100
Completed batch: 1200
Completed batch: 1300
Completed batch: 1400
Completed batch: 1500
Completed batch: 1600
Completed batch: 1700
Completed batch: 1800
Completed batch: 1900
Completed batch: 2000
Completed batch: 2100
Completed batch: 2200
Completed batch: 2300
Completed batch: 2400
Completed batch: 2500
Completed batch: 2600
Completed batch: 2700
Completed batch: 2800
Completed batch: 2900
Completed batch: 3000
Completed batch: 3100
Completed batch: 3200
Completed batch: 3300
Completed batch: 3400
Completed batch: 3500
Completed batch: 3600
Completed batch: 3700
Completed batch: 3800
Completed batch: 3900
Completed batch: 4000
Completed batch: 4100
Completed batch: 4200
Completed batch: 4300
Completed batch: 4400
Completed batch: 4500


In [92]:
c = requires_grad_standing("COMPSCI, ECE 760", prereq_map, text_prereqs)
print(c)

1 True
True


In [135]:
for update in updates:
    if not update["needs_grad_standing"]:
        course = update["course_name"]
        if "861" in course:
            prereq_text = text_prereqs.get(course, "")
            print(f"{course} → grad-only\n  Prereq text: {prereq_text}\n")


CS&D 861 → grad-only
  Prereq text: Declared in Audiology Consortial Program with UW-Stevens Point AUD

SOCWORK 861 → grad-only
  Prereq text: Declared in Social Work MSW

COMPSCI, STAT, ECE 861 → grad-only
  Prereq text: E C E/​COMP SCI  761orE C E 830

HISTORY 861 → grad-only
  Prereq text: Consent of instructor

